In [5]:
model_name = "my500_neptuneselected_nometa"

In [6]:
import numpy as np
import pandas as pd
import gc
from time import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

In [7]:
timesheet = [time()]
def timer(statement):
    global timesheet
    timesheet.append(time())
    print statement+" :", (timesheet[-1]-timesheet[-2]),"seconds"
timer("Init...")

Init... : 0.000298976898193 seconds


In [8]:
def load_meta(directory, prefix, modeltype):
    m_tr = pd.read_csv(directory+prefix+"_train.csv")
    m_te = pd.read_csv(directory+prefix+"_test.csv")
    data = pd.concat([m_tr, m_te], axis=0).reset_index(drop=True)
    data.columns = ["{}_{}_{}".format(c, prefix, modeltype) if c!="SK_ID_CURR" else c for c in data.columns]
    return data

def join_features(data, features):
    for item in features:
        data = data.merge(item, how = "left", on = "SK_ID_CURR")
    return data

def load_data(datafile):
    global important_columns
    colnames = [c.replace(" ","_") for c in pd.read_csv(datafile, nrows= 1).columns]
    intersection = list(set(colnames).intersection(set(important_columns)))+["SK_ID_CURR"]

    df =  pd.read_csv(datafile, names = colnames, usecols = intersection, skiprows=1).dropna(axis=1, how="all")
    timer("Loaded {} with shape {} in ".format(datafile.split("/")[-1], df.shape))
    return df

## Getting Feature Importance File

In [9]:
important_columns = pd.read_csv("../feature selector/importance/shap_importances.csv")
important_columns["feature"] = important_columns["feature"].apply(lambda x: x.replace(" ","_"))
important_columns = important_columns.loc[important_columns.shapely_mean > 0.0].reset_index(drop=True)
important_columns = important_columns.sort_values(by = "shapely_mean", ascending = False).reset_index(drop=True)
important_columns = important_columns.loc[0:500].feature.values.tolist()

## Loading Features

In [10]:
applications = load_data("../extractor/csv/application_features_V2.csv")
bureau_balance_bb = load_data("../extractor/csv/bureau_features_V1.csv")
credit_card_balance = load_data("../extractor/csv/credit_card_features_V1.csv")
installment_features = load_data("../extractor/csv/installment_features_V1.csv")
pos_cash_balance = load_data("../extractor/csv/pos_cash_features_V1.csv")
previous_apps = load_data("../extractor/csv/previous_application_features_V1.csv")

credit_card_balance_v3 = load_data("../extractor/csv/credit_card_features_V3.csv")
installment_features_v3 = load_data("../extractor/csv/installment_features_V3.csv")
pos_cash_balance_v3 = load_data("../extractor/csv/pos_cash_features_V3.csv")
previous_apps_v3 = load_data("../extractor/csv/previous_application_features_V3.csv")

base_features = [applications, bureau_balance_bb, credit_card_balance, installment_features, pos_cash_balance,previous_apps, credit_card_balance_v3,installment_features_v3,pos_cash_balance_v3, previous_apps_v3]

Loaded application_features_V2.csv with shape (356255, 75) in  : 24.7878141403 seconds
Loaded bureau_features_V1.csv with shape (305811, 84) in  : 10.3421669006 seconds
Loaded credit_card_features_V1.csv with shape (103558, 16) in  : 10.3835639954 seconds
Loaded installment_features_V1.csv with shape (339587, 100) in  : 33.1001639366 seconds
Loaded pos_cash_features_V1.csv with shape (337252, 48) in  : 21.0373461246 seconds
Loaded previous_application_features_V1.csv with shape (338857, 93) in  : 22.048017025 seconds
Loaded credit_card_features_V3.csv with shape (103558, 6) in  : 16.998759985 seconds
Loaded installment_features_V3.csv with shape (339587, 69) in  : 27.6496949196 seconds
Loaded pos_cash_features_V3.csv with shape (337252, 11) in  : 6.99699497223 seconds
Loaded previous_application_features_V3.csv with shape (338857, 9) in  : 8.45656704903 seconds


## Joining All Files

In [11]:
train = pd.read_csv("../data/application_train.csv", usecols = ["SK_ID_CURR","TARGET"])
test = pd.read_csv("../data/application_test.csv", usecols = ["SK_ID_CURR"])

neptune_train = pd.read_csv("../neptune extractor/data/train.csv")
neptune_test = pd.read_csv("../neptune extractor/data/test.csv")

train = pd.concat([train, neptune_train], axis=1)
test = pd.concat([test, neptune_test], axis=1)

del neptune_train, neptune_test
gc.collect()

data = pd.concat([train, test], axis=0).reset_index(drop=True)
data = join_features(data, base_features)
del base_features,applications, bureau_balance_bb, credit_card_balance, installment_features, pos_cash_balance,previous_apps, credit_card_balance_v3,installment_features_v3,pos_cash_balance_v3, previous_apps_v3
gc.collect()

train = data.loc[data.TARGET.notnull()].reset_index(drop=True)
test = data.loc[data.TARGET.isnull()].reset_index(drop=True)

train_id = train[["SK_ID_CURR"]]
test_id = test[["SK_ID_CURR"]]
test_id_rank = test[["SK_ID_CURR"]]
target =train.TARGET

train.drop(["SK_ID_CURR", "TARGET"], axis=1, inplace=True)
test.drop(["SK_ID_CURR","TARGET"], axis=1, inplace = True)

# Defining Model

In [15]:
import lightgbm as lgb
def model_tree(x_train, x_test, y_train, y_test, test, meta_train, meta_test,train_index, test_index,fold_id):
    dtrain = lgb.Dataset(x_train, label=y_train)
    dval = lgb.Dataset(x_test, label=y_test)
    params = {
        'num_leaves': 33, #32
        'reg_alpha': 0.0, 
        'colsample_bytree': 0.05, 
        'subsample_freq': 1, 
        'learning_rate': 0.02, # 0.02
        'boosting_type': 'gbdt', 
        'nthread': 8, 
        'min_split_gain': 0.5, 
        'n_estimators': 10000, 
        'subsample': 1, 
        'reg_lambda': 100, 
        'objective': "binary", 
        'min_child_samples': 70, 
        'max_depth': -1,
        'class_weight': None,
        "bagging_seed" : 3143,
        "seed":1343,
        "metric":"auc",
        "is_unbalance": False,
        "scale_pos_weight": 1,
        "max_bin":300
    }
    model = lgb.train(params, dtrain, num_boost_round=5000,valid_sets=[dtrain, dval], early_stopping_rounds=200, verbose_eval=100)
    meta_train[test_index] = model.predict(x_test, num_iteration=model.best_iteration or 5000)
    meta_test.append(model.predict(test, num_iteration=model.best_iteration or 5000))
    
    global fold_roc
    fold_roc.append(roc_auc_score(y_test, meta_train[test_index]))
    # Calculate Feature Importance
    global feature_importance
    gain = model.feature_importance('gain')
    fold_feature_importance = pd.DataFrame({'feature':model.feature_name(), 'split':model.feature_importance('split'), 'gain':100 * gain / gain.sum()})
    feature_importance = feature_importance.append(fold_feature_importance, ignore_index=True) 

# Training Model

In [16]:
meta_train = np.zeros(train.shape[0])
meta_test = []
feature_importance = pd.DataFrame(columns = ["feature","split","gain"])
fold_roc = []

kf = StratifiedKFold(n_splits= 10, shuffle=True, random_state=47)
for fold_id, (train_index, test_index) in enumerate(kf.split(train, target)):
    x_train, x_test = train.iloc[train_index], train.iloc[test_index]
    y_train, y_test = target[train_index], target[test_index]

    model_tree(x_train, x_test, y_train, y_test, test, meta_train, meta_test,train_index, test_index,fold_id)

test_id["TARGET"] = np.array(meta_test).T.mean(axis=1)
test_id_rank["TARGET"] = pd.DataFrame(np.array(meta_test).T).rank(pct = True).mean(axis=1)
train_id["TARGET"] = meta_train

print "Overall ROC: {},  Mean ROC: {}, STD AUC: {}".format(roc_auc_score(target, meta_train), np.mean(fold_roc), np.std(fold_roc))

Training until validation scores don't improve for 200 rounds.
[100]	training's auc: 0.759236	valid_1's auc: 0.744977
[200]	training's auc: 0.779809	valid_1's auc: 0.761227
[300]	training's auc: 0.794556	valid_1's auc: 0.771992
[400]	training's auc: 0.805002	valid_1's auc: 0.778171
[500]	training's auc: 0.813239	valid_1's auc: 0.781614
[600]	training's auc: 0.819921	valid_1's auc: 0.783981
[700]	training's auc: 0.825991	valid_1's auc: 0.785991
[800]	training's auc: 0.831621	valid_1's auc: 0.787487
[900]	training's auc: 0.836706	valid_1's auc: 0.788569
[1000]	training's auc: 0.841483	valid_1's auc: 0.78945
[1100]	training's auc: 0.845933	valid_1's auc: 0.790148
[1200]	training's auc: 0.850201	valid_1's auc: 0.790694
[1300]	training's auc: 0.854155	valid_1's auc: 0.791304
[1400]	training's auc: 0.857947	valid_1's auc: 0.791584
[1500]	training's auc: 0.8615	valid_1's auc: 0.79188
[1600]	training's auc: 0.864859	valid_1's auc: 0.792145
[1700]	training's auc: 0.868189	valid_1's auc: 0.79230

[1100]	training's auc: 0.845624	valid_1's auc: 0.79367
[1200]	training's auc: 0.849909	valid_1's auc: 0.794083
[1300]	training's auc: 0.85386	valid_1's auc: 0.794527
[1400]	training's auc: 0.857671	valid_1's auc: 0.794839
[1500]	training's auc: 0.861163	valid_1's auc: 0.795005
[1600]	training's auc: 0.864585	valid_1's auc: 0.795212
[1700]	training's auc: 0.867843	valid_1's auc: 0.795465
[1800]	training's auc: 0.871015	valid_1's auc: 0.795487
[1900]	training's auc: 0.874094	valid_1's auc: 0.795598
[2000]	training's auc: 0.876974	valid_1's auc: 0.795717
[2100]	training's auc: 0.879781	valid_1's auc: 0.79581
[2200]	training's auc: 0.882497	valid_1's auc: 0.795865
[2300]	training's auc: 0.88517	valid_1's auc: 0.79579
Early stopping, best iteration is:
[2175]	training's auc: 0.881838	valid_1's auc: 0.795885
Training until validation scores don't improve for 200 rounds.
[100]	training's auc: 0.759237	valid_1's auc: 0.743786
[200]	training's auc: 0.779861	valid_1's auc: 0.759433
[300]	trainin

[900]	training's auc: 0.835877	valid_1's auc: 0.796189
[1000]	training's auc: 0.840704	valid_1's auc: 0.797103
[1100]	training's auc: 0.845153	valid_1's auc: 0.797964
[1200]	training's auc: 0.849422	valid_1's auc: 0.798701
[1300]	training's auc: 0.853386	valid_1's auc: 0.799213
[1400]	training's auc: 0.857156	valid_1's auc: 0.799719
[1500]	training's auc: 0.860868	valid_1's auc: 0.799995
[1600]	training's auc: 0.864251	valid_1's auc: 0.800287
[1700]	training's auc: 0.867505	valid_1's auc: 0.800478
[1800]	training's auc: 0.87076	valid_1's auc: 0.80074
[1900]	training's auc: 0.873833	valid_1's auc: 0.800935
[2000]	training's auc: 0.876696	valid_1's auc: 0.801205
[2100]	training's auc: 0.879536	valid_1's auc: 0.801235
[2200]	training's auc: 0.882295	valid_1's auc: 0.80135
[2300]	training's auc: 0.884937	valid_1's auc: 0.801395
[2400]	training's auc: 0.887547	valid_1's auc: 0.801495
[2500]	training's auc: 0.890091	valid_1's auc: 0.801712
[2600]	training's auc: 0.892428	valid_1's auc: 0.801

/home/sakhtar0092/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/sakhtar0092/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/sakhtar0092/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

Overall ROC: 0.79841785361,  Mean ROC: 0.798425216237, STD AUC: 0.00540273720633


In [17]:
fold_roc

[0.7938222724673166,
 0.8043878037427296,
 0.8014559566087992,
 0.8040982547750826,
 0.7958851022853384,
 0.789520138759014,
 0.8056169269454663,
 0.7924721643776325,
 0.7952668296209857,
 0.801726712790561]

In [18]:
train_id.to_csv("csv/{}_train.csv".format(model_name), index=False)
test_id.to_csv("csv/{}_test.csv".format(model_name), index=False)
test_id_rank.to_csv("csv/{}_rank_test.csv".format(model_name), index=False)